In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from glob import glob
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tqdm.keras import TqdmCallback  
import datetime


class LipReadingDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_path, alignment_path, batch_size=32, frame_length=75, 
                 image_height=46, image_width=140, **kwargs):
        super().__init__(**kwargs)
        self.data_path = data_path
        self.alignment_path = alignment_path
        self.batch_size = batch_size
        self.frame_length = frame_length
        self.image_height = image_height
        self.image_width = image_width

        self.video_paths = sorted(glob(os.path.join(data_path, '*.mpg')))
        self.alignment_paths = sorted(glob(os.path.join(alignment_path, '*.align')))
        
        print(f"Found {len(self.video_paths)} video files and {len(self.alignment_paths)} alignment files")
        self.vocabulary = self._create_word_vocabulary()
            
        self.char_to_num = tf.keras.layers.StringLookup(
            vocabulary=self.vocabulary, oov_token="")
        self.num_to_char = tf.keras.layers.StringLookup(
            vocabulary=self.vocabulary, oov_token="", invert=True)
    
    def _create_word_vocabulary(self):
        words = set()
        print(f"Processing alignment files from: {self.alignment_path}")
        
        for align_path in self.alignment_paths:
            try:
                with open(align_path, 'r') as f:
                    content = f.read().strip().split()
                    words.update([content[i] for i in range(2, len(content), 3)])
            except Exception as e:
                print(f"Error processing {align_path}: {str(e)}")
        
        words.discard('sil')
        vocabulary = sorted(list(words))
        
        if not vocabulary:
            print("No words found in alignment files. Using default vocabulary.")
            vocabulary = ['bin', 'blue', 'at', 'f', 'two', 'now']
        
        print(f"Vocabulary size: {len(vocabulary)}")
        return vocabulary
    
    def __len__(self):
        return max(1, len(self.video_paths) // self.batch_size)
    
    def _process_video(self, video_path):
        frames = []
        cap = cv2.VideoCapture(video_path)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            mouth = gray[190:236, 80:220]
            mouth = cv2.resize(mouth, (self.image_width, self.image_height))
            frames.append(mouth)
        
        cap.release()
        
        frames = np.array(frames, dtype=np.float32)
        frames = (frames - frames.mean()) / (frames.std() + 1e-6)
        
        if len(frames) < self.frame_length:
            pad_length = self.frame_length - len(frames)
            frames = np.pad(frames, ((0, pad_length), (0, 0), (0, 0)), mode='constant')
        else:
            frames = frames[:self.frame_length]
        
        return frames
    
    def _process_alignment(self, alignment_path):
        with open(alignment_path, 'r') as f:
            content = f.read().strip().split()
        
        words = [content[i] for i in range(2, len(content), 3) if content[i] != 'sil']
        text = ' '.join(words)
        return self.char_to_num(tf.convert_to_tensor(text.split()))
    
    def __getitem__(self, idx):
        batch_videos = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_alignments = self.alignment_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        X = np.zeros((len(batch_videos), self.frame_length, self.image_height, self.image_width, 1))
        Y = np.zeros((len(batch_videos), len(self.vocabulary)))
        
        for i, (video_path, align_path) in enumerate(zip(batch_videos, batch_alignments)):
            frames = self._process_video(video_path)
            X[i] = frames.reshape(self.frame_length, self.image_height, self.image_width, 1)
            
            labels = self._process_alignment(align_path)
            Y[i] = tf.reduce_max(tf.one_hot(labels, len(self.vocabulary)), axis=0)
        
        return X, Y


def build_model(frame_length, image_height, image_width, vocabulary_size):
    model = Sequential([
        tf.keras.Input(shape=(frame_length, image_height, image_width, 1)), 
        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Conv3D(128, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Conv3D(256, kernel_size=(3, 3, 3), activation='relu'),
        MaxPool3D(pool_size=(1, 2, 2)),
        BatchNormalization(),
        
        Reshape((-1, 256)),
        
        Bidirectional(LSTM(128, return_sequences=True)),
        Dropout(0.5),
        
        Bidirectional(LSTM(64)),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(vocabulary_size, activation='softmax')
    ])
    
    return model


def train_and_save_main_model(data_dir, alignment_dir, batch_size=32):
    print("Starting training for sentence-level prediction...")
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model_dir = f"models_main_{timestamp}"
    os.makedirs(model_dir, exist_ok=True)
    
    data_generator = LipReadingDataGenerator(data_dir, alignment_dir, batch_size=batch_size)
    
    model = build_model(
        frame_length=75,
        image_height=46,
        image_width=140,
        vocabulary_size=len(data_generator.vocabulary)
    )
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        ModelCheckpoint(
            os.path.join(model_dir, 'lip_reading_main_best.keras'),
            save_best_only=True,
            monitor='accuracy'
        ),
        EarlyStopping(
            monitor='loss',
            patience=10,
            restore_best_weights=True
        ),
        TqdmCallback(verbose=1)
    ]

    print("Training started...")
    model.fit(
        data_generator,
        epochs=10,
        callbacks=callbacks
    )

    final_model_path = os.path.join(model_dir, 'model.h5')
    model.save(final_model_path)
    print(f"Final model saved: {final_model_path}")

    vocab_path = os.path.join(model_dir, 'vocabulary_main.txt')
    with open(vocab_path, 'w') as f:
        f.write('\n'.join(data_generator.vocabulary))
    print(f"Vocabulary saved: {vocab_path}")

    return model, model_dir


In [ ]:
def predict_with_main_model(model_dir, video_path):
    vocab_path = os.path.join(model_dir, 'vocabulary_main.txt')
    with open(vocab_path, 'r') as f:
        vocabulary = f.read().splitlines()

    data_generator = LipReadingDataGenerator("", "")
    data_generator.vocabulary = vocabulary
    data_generator.char_to_num = tf.keras.layers.StringLookup(
        vocabulary=vocabulary, oov_token="")
    data_generator.num_to_char = tf.keras.layers.StringLookup(
        vocabulary=vocabulary, oov_token="", invert=True)

    model = load_model(os.path.join(model_dir, 'model.h5'))

    frames = data_generator._process_video(video_path)
    frames = frames.reshape(1, data_generator.frame_length, 
                            data_generator.image_height, 
                            data_generator.image_width, 1)

    prediction = model.predict(frames)
    predicted_indices = tf.argmax(prediction, axis=1)
    predicted_text = data_generator.num_to_char(predicted_indices)

    return ' '.join(predicted_text.numpy().decode('utf-8').split())


if __name__ == "__main__":
    data_dir = r"data/s1"
    alignment_dir = r"data/alignments/s1"

    print("Training main model...")
    main_model, main_model_dir = train_and_save_main_model(data_dir, alignment_dir)

    test_video = r"data\s1\bbaf2n.mpg"

    print("\nMaking predictions...")
    sentence_prediction = predict_with_main_model(main_model_dir, test_video)
    print(f"Predicted sentence: {sentence_prediction}")


Training main model...
Starting training for sentence-level prediction...
Found 1000 video files and 1000 alignment files
Processing alignment files from: data/alignments/s1
Vocabulary size: 52


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Training started...
Epoch 1/10
